# Build your own Gillespie algorithm 


In this tutorial you will learn in detail about the ~~deterministic~~ *stochastic* formalism for the *temporal* modelling of *simple*\* chemical systems, and you will learn to write the *Gillespie algorithm*, from scratch, using Python 2.7.

\* the word *simple* referst to "spatially homogenous" (i.e. spatially uniform mixture of chemical species, as opposed to spatially varied, where you could find regions with higher concentrations of some molecules but not others).

As you can already imagine, there are many Gillespie algorithm scripts out there, available for you to download freely and use for your simulations. (In particular modifications of the Gillespie algorithm such as the reaction method (Gibson & Bruck) and the tau-leaping are very efficient.) We think it is advantageous in many ways to understand how this formalism is constructed. It will help you, for example, decide in what contexts this framework is more appropiate, and understand the reasons why you might get different results with the deterministic and stochastic approaches.


### To note

If you would like to learn in detail about the *deterministic* framework for modelling the temporal evolution of simple chemical systems, we refer you to [this](https://github.com/karinsasaki/biomath-modelling-with-python) tutorial.

The tutorial you currently hold deals only with modelling the evolution of a chemical system, in *time*. If you would like to learn to desing a *spatial* model and simulate it, we refer you to [this (coming soon)]() tutorial.


### Structure of tutorial

In **Part 1** we introduce and characterise the stochastic framework as a mathematical model of simple systems of chemical reactions.

In **Part 2** we set the problem that needs to be solved, in precise mathematical formulation.

In **Part 3** we provide steps to help you build your own Gillespie agorithm.


### References

The tutorial is heavily based on Gillespie's original 1977 [paper](http://wwwf.imperial.ac.uk/~nsjones/gillespie_1977.pdf) and we will often  quote text directly from the paper. 

The Wikipedia [page](https://en.wikipedia.org/wiki/Mathematical_model) on Mathematical Models.


### Acknowledgements

This tutorial was make by Dr. Karin Sasaki (Centre for Biological Modelling at the European Molecular Biology Laboratory).

## Part 1. Characterisation of stochastic mathematical models of the time behaviour of  spatially homogenous chemical systems.

### The *stochastic approach* or framework:

1.  The time evolution is regarded as a kind of random-walk process. (A *deterministic model*, in contrast, regards the time evolution as a continuous, wholly predictable process.)
2. The time evolution of the system is governed by a single differential-difference equation called the “master equation” or is computationally calculated exactly by the Gillespie algorithm. (In a *deterministic model*, the time evolution of the system is governed by a set of coupled, ordinary differential equations called the “reaction-rate equations”.)


### Reasoning behind the stochastic framework

1. The time evolution of a chemically reacting system is not a continuous process, because molecular population levels obviously can change only by discrete integer amounts. 
2. The time evolution is not a deterministic process, for, the molecular motions are regarded to be governed by the equations of classical mechanics, it is impossible to predict the exact molecular population levels at some future time, unless we take account of the precise positions and velocities of all the molecules in the system. In other words, although the temporal behavior of a chemically reacting system of classical molecules is a deterministic process in the full position-momentum phase space of the system, it is not a deterministic process in the N-dimensional subspace of the species population numbers.


### Classification of stochastic models as mathematical models

* **The stochastic model is linear (not nonlinear)** - because chemical reactions exihibit linear relationships between the reactants. For example, an enzymatic reaction 
    
    S + E <-> SE -> E + P
    
is broken down into

    S + E -> SE
    SE -> S + E
    SE -> E + P
    
which all have linear relations between the reactants and products. Note that a linear model does not necessarily predic non-linear expressions.

* **Stochastic models are dynamic (not static i.e. time-invariant)** -  because they accounts for time-dependent changes in the state of the system. (A *static model* calculates the system in equillibrium. An example is Flux Balance Analysis of metabolic models using linear programming).

* **Stochastic models are explicit** -  all of the input parameters of the overall model are known, and the output parameters can be calculated by a finite series of computations 

* **Stochastic models are discrete** - objects are treated as discrete entities. (A *continuous model* represents the objects in a continuous manner, for example, chemical reactions can be regarded as a continuous rate processes).

* **Stochastic models are probabilistic and not deterministic** - randomness is included in the model in order to capture inherent fluctuations in the molecular population levels. Therefore variable states are described by probability distributions and these models can perform differently for the same set of initial conditions. (A deterministic model is one where the variable states are uniquely determined by parameters in the model and by sets of previous states of these variables. Therefore, a deterministic model always performs the same way for a given set of initial conditions.)

# Part 2. Setting the problem and translating it into mathematical formalism

### The problem we want to solve

The system:

M chemical reactions `R1, ... ,RM` that involve `N` chemical species `S1, ..., SN`.  

The question:

Given `X1, ..., XN` number of particles for each of the species, at some initial time, what will these molecular population levels be at a later time?


### Modelling the system stochastically - Translating of the problem into stochastic language 

#### Assumptions of the stochastic framework

1. A fixed volume `V` 
2. `V` contains a spatially uniform mixture of `N` chemical species
3. The chemical species are in thermal (but not necessarily chemical) equilibrium, which means that collisions occur in an essentially random manner.
3. A chemical reaction occurs whenever two or more molecules of "appropriate kinds" (defined by the reaction equations) collide in an "appropriate way" (assuming that the two molecules `S1` and `S2` are hard spheres of radii `rl` and `r2`, a collision will occur between them whenever the center-to-center distance between the two molecules decreases to `r12= r1 + r2`). 
4. All collisions are reactive, i.e. they result in chemical alterations of the colliding molecules as defined by the reaction equations.


#### A simple chemical system modelled stochastically is characterised by a "collision/reaction probability per unit time" (i.e. it is a stochastic Markov process) and not by a "collision rate" (i.e. a deterministic rate process)

**Side note:** A stochastic Markov process is one that satisfies the so called *Markov property*, which says that the conditional probability distribution of future states of a process depends only upon the present state, not on the sequence of events that preceded it. (Compare this with the deterministic framework which assumes that the any future state of the system is determined by previous states and specifically the initial state of the system.)

Suppose we have two molecules `S1` and `S2`. Whilst we cannot rigorously calculate *the number of collisions* between `S1` and `S2`, occurring in `V` in any infinitesimal time interval, we can rigorously calculate *the probability of a collision* occurring between `S1` and `S2`, in `V` in any infinitesimal time interval:

#### Calculating the propensity of each reaction (the collision/reaction probability per unit time)

Let's begin with an **example**. Suppose the `S1` and `S2` molecules in `V` can undergo the reaction

```
R1: S1 + S2 -> 2S1
```

Then we may assert the existence of a constant `cl`, which depends only on the physical properties of the two molecules and the temperature of the system, such that `c1*dt` is *the average probability that a particular `S1-S2` molecular pair will react according to `R1` in the next infinitesimal time interval `dt`*.

This implies that, if at time `t` there are `X1` of the `S1` molecules and `X2` of the `S2` molecules, making a total of `h1 = Xl*X2` distinct `S1-S2` pairs. Let `a1=h1*c1` then `a1*dt` is *the probability that an `R1` reaction will occur somewhere inside `V` in the next infinitesimal time interval `(t, t + dt)`*.

<img src="diag.png" style="width: 400px;"/>

Let us take another example. Consider the reverse of the reaction of `R1`:

```
R2: 2S1 -> S1 + S2
```

Then we would characterize this reaction by a constant `c2`, such that `c2*dt` is the average probability that a particular pair of `S1` molecules will react according to `R2` in the next `dt`. However, the number of distinct pairs of `S1` molecules in `V` is not `X1*X1` but `h2 = X1*(X1-1)/2!`. . Let `a2=h2*c2`. Hence, the probability that an `R2` reaction will occur somewhere inside `V` in the next time interval `dt` is `a2*dt`. 

Now, for the **general form**, suppose the volume `V` contains a mixture of `Xj` molecules of chemical species `Sj (j = 1, ..., N)` and suppose further that these `N` species can interreact through `M` specified chemical reaction channels `Ri (i = 1, ..., M)`. Then we may assert the existence of `M` constants `ci (i =  1, ..., M)`, which depend only on the physical properties of the molecules and the temperature of the system, such that `ci*dt` is the *the average probability that a particular combination of `Ri` reactant molecules will react accordingly in the next infinitesimal time interval dt*. 

Let `hi` be the total number of distinct combinations of `R_i` reactant molecules in `V` at time `t` (a simple algebraic expression calculated using the binomial coefficient of the stoichiometry of the reactants). Then, if we multiply `ci*dt` by `hi`, we will obtain the probability `ai*dt` (where `ai=hi*ci`) that an `Ri` reaction will occur somewhere inside `V` in the next infinitesimal time interval `(t, t + dt)`.

**In summary**:
- *the propensity of a reaction* `Ri (i = 1, ..., M)` is governed solely by the fundamental hypothesis that, for  constants `ci`, which depends only on the physical properties of the reactants of `Ri`, `ci*dt` is *the average probability that the particular combination of `Ri` reactant molecules will react in the next infinitesimal time interval dt*. 
- furthermore, the probability of reaction `Ri` happening in the next time interval `(t, t + dt)` is calculated as the product `hi` of the total number of distinct combinations of `Ri` reactant molecules and `ci*dt`, i.e. `ai*dt` for `ai=hi*ci`.



#### Calculating the time evolution of the system (that is modelled stochastically)

Let us consider how to sample the time evolution of this system in a way that follows the probability density function of the Chemical Master Equation. Remember that the system is a Markov process, so it is enough to base the argument on the current state of the system. 

If we are given that the system is in the state `(X1,...,XN)` at time `t`,  essentially all we need in order to “move the system forward in time” is to answer these two questions: 

1. when will the next reaction occur, and 
2. what kind of reaction will it be 

One way to obtain the next time `T` and reaction `mu`, could be to draw two random numbers `r1` and `r2` from the normal distribution and to use them to calculate the next time and reaction with the following formulas:

<img src="c2.png" style="width: 180px;"/>

<img src="a0.png" style="width: 350px;"/>

<img src="a.png" style="width: 100px;"/>

(Note that this relies on the propensities of the reactions that we calculated just above; in other words, the next reaction and time are influenced by the propencities of the reactions, but this make sense, since a reaction with a higher propensity should have a higher probabilty of happening than one with a lower propensity.)

It is shown in Gillespie's paper that choosing the next time and reaction in this way, samples directly from the *probability density function* that is the Chemical Master Equation. (Note that this probability density function is a joint probability density function (also knows as a probability measure) of the continuous variable T (0<= T < inf), for the time of the next reaction, and the discrete variable (mu = 1, 2,. ..,M).)

So, in practice, according to the calculations we have made above, of the reation propensities and the next time and reaction, to simulate a stochastic model of any simple chemical system you can just follow this algorithm (the Gillespie algorithm) below:

<img src="b.png" style="height: 400px;"/>

and you already have all the tools you need to calculate each step.

## Part 3. The stochastic simulation algorithm

Now that we have the algorithm, we can program it into python so that we can then use it to simulate any simple chemical system. The rest of the tutorial is a collection of small exercises that together lead you to program your own Gillespie algorithm. The instuctions are given in human/maths language and you need to find the appropiate python commands to achieve those steps. Don't worry, we give you hint and you can always refer to the documentation.


### Part 3.A. Let's go through one loop of the algorithm, each step (sort of) by hand

We will use the following example system:

```
R1: S1 + S2 -> 2S1
```

For each of the steps, follow the instructions and hints and if necessary, check the python documentation.

**Import the required libraries and modules**

In [1]:
# import modules and libraries: numpy, factorial from scipy.special, pyplot from matplotlib
# allow inline plot with %matplotlib inline

**Step 0 - Initialization:** 
- Input the desired values for the `M` reaction constants `c1,. ..,cM` and the `N` initial molecular population numbers `X1,...,XN`. 
- Set the time variable `t` and the reaction counter `n` both to zero. 

In [ ]:
# reaction constants, c1

print "c1: ", c1

# initial molecular population numbers X1 of S1 and X2 of S2, 

print "X1: ", X1
print "X2: ", X2

# initialise time variable t and reaction counter n

print "t: ", t
print "n: ", n

**Step 1 - calculate the ai and a0: ** 
- Calculate and store the `M` quantities `a1 = h1*c1,..., aM = hm*cm` for the current molecular population numbers. 
- To fasciliate the manipulation with python, save all the `ai`'s in an array `a`.
- Calculate and store as `a_0` the sum of the `M` `a_i` values.

In [ ]:
# calculate a1 = h1*c1 where h1 is the total number of all possible 
# combinations of pairs S1 and S2
    # go back to the section where the propensities of the reactions were calculated to remind yourself 

# define an (numpy) array a with entries each ai

# define a0 as the sum of all ai's

# debug
print "a1: ", a1
print "a: ", a
print "a0: ", a0

**Step 2 - Get the next reaction and time of reaction: ** 

- Generate the pair `(T, mu)` from the set of random pairs whose probability density function is the Chemical Master Equation.
- When calculating the next reaction, you need a while loop.
- Recall that numbering in Python starts from 0, so be careful with indexing!

In [ ]:
# generate random numbers r1 and r2 from the normal distribution, using np.random.random

print "r1: ", r1
print "r2: ", r2

# find the increment T in time as (1/a0)*ln(1/r1)

print "T: ", T

# choose next reaction
    # initialise mu to 0
    
    # define new variable N = r2*a0 - a[mu]

    # while N is positive
        
        # add 1 to mu
        
        # substract a_mu from N
    
    # when the while lopp breaks, allocate the current value of mu to the next reaction variable next_r

print "next_r: ", next_r

Does it make sense what the next reaction is?

What happens if you run the code again? Does the same reaction get chosen? Does this make sense?

Make sure that you understand how this while loop achieves the purpose of choosing the next reaction, correctly according to the theory given above, in section "Calculating the time evolution of the system".

**Step 3 - update the system: ** 

Using the `T` and `mu` values obtained in step 2, increase `t` by `T`, and adjust the molecular population levels to reflect the occurrence of one `R_i` reaction; e.g., if `R_i` is the reaction `R1`, then increase `X1` by `1` and decrease `X2` by `1`. Then increase the reaction counter `n` by `1`.

(Note that it is enough to recalculate only those quantities `a_i` corresponding to reactions `R_i` whose reactant population levels were just altered in step 3; also, `a_0` may be recalculated simply by adding to `a_0` the difference between each newly changed `a_i` value and its corresponding old value.)

In [ ]:
# define the time of next reaction

print "t: ", t

# add one to the number of reactions count n

print "n: ", n

# update the system according to the reaction that was chosen

print "X1: ", X1
print "X2: ", X2

We have simulated the stochastic occurance of one reaction. We can visualise the effect of this reaction by plotting the time vs the concentration of the reactants as follows:

In [ ]:
# create arrays containing all data

    # array with initial number of X1 and number after one reaction
    
    # array with initial number of X2 and number after one reaction
    
    # array with initial time and time of reaction one

# create plot using pyplot, remeber to add a legend and annotate the axes


Return to 1 and repeat.

### Part 3.B. - Automate the process

So now, we need to do the same process many many times, to simulate the system to a maximum time (`tmax`) or maximum number of reactions (`nmax`) of our choice. 

It is not feasible to carry on typing the same commands every time we need to go through the loop until we reach either `tmax` or `nmax` and we need to implement an automated way of **looping** through the algorithm.

So what should we use? You've guessed it - a while loop, because with a while loop we can test whether the `tmax` or `nmax` bounds have been reached and *while* they have not, we go through yet another loop of the algorithm.

Additionally, to be able to visualise the output of every loop, we will also want to provide an automated way of storing the changing values of the reactants, rather than doing it by hand as we did above.

Finally, it is always a good practice to implement a debugging strategy. We recommend defining a *pause* function that will ask the user (of the program) to press the `<ENTER>` key to continue and to print values that will help the user keep track of the algorithm.

To implement these new changes, let's use a new biochemical system, that has more than one reaction. It is important that this system has more than one reaction because if you can write a program general enough to deal with more than one reaction, then there should be no problem with a system with more reactions. Let's use the following system:
```
R1: 2X1 + X2 -> X3
R2: X1 -> 0
```

#### Import required libraries

In [ ]:
# import required libraries and modules: numpy, factorial from scipy.special, pyplot 
# enable inline plotting

#### Define debug function

In [ ]:
# define a pause function for debugging

#def ...
    
    # using the command raw_input, define a variable that prints out a message along the lines of 
    # "Press the <ENTER> key to continue..." and take as input nothing.


#### Step 0 of algorithm

In [ ]:
# ****************************   
# step 0: input rate values, initial contidions values and initialise time 
# and reactions counter
# ****************************   

# Define the stoichiometry of the system as a numpy array, with the number of reactions in the rows 
# and the number of reactants in the columns. 

    # Define the stochiometry of the substrates and products separatelly; Use the variable names 
    # stoch_subst for the substrates and stoch_prods for the products
    
    # Using these define the stoch variable as the summ of stoch_subst and stoch_prods

    # Keep track of the following:
    # (i ranges 0 -> M-1, where M is the number of reactions, j ranges 0 -> N-1, where N is the 
    # number of species)

    # allocate the number of reactions to variable num_rxn and number of species to varialbe num_spec,
    # use the command shape of numpy and array indexing

# define the ci parameters in an array variable called rates

# define the initial conditions of the reactants in an array variable init

# specify the maximum time, tmax, and maximum number of reactions, nrmax

# Initialise the current_species variable to init

# Initialise the current time current_t and the time and reaction counters t_count 
# and react_count to 0

# debug: print all the variables to check everything is correct

In [ ]:
# Initialise variables to store time and molecule numbers

    # You need the arrays to be large enough to contain all the possible number of iterations the 
    # algorithm goes through (which we don't know, a priori, how many iterations that will be). 
    # Call this cariable largenum

    # Initialise a zeros array called store_t to store all the times of reactions (with dimensions largenum,1)
    
    # Initialise a zeros array called store_mols to store all reactants after each reaction 
    # (with dimensions largenum,num_spec)

    # Initialise a zeros array called store_r to store all the reaction ids (with dimensions largenum,1)

# debug by printing all the variables and check for correct numbers

In [ ]:
# store current time and state

    # store current time of the system by indexing store_t with t_count (why?) and equating that entry it 
    # to current_t

    # store current numbers of molecules of each species by indexing store_mols with [t_count,:] (why?) and 
    # equating that row it to current_species

# debug by printing 10 rows of the two arrays above and checking for correct recording of data


#### Define a function to automate the choosing of the next time and reaction and check it works correctly

In [2]:
# define a function to choose the next time and reaction

# define function with name choose_t_r and inputs a0 and a
    
    # generate random numbers r1 and r2 from the uniform distribution
    
    # choose the increment T in time as (1/a0)*ln(1/r1)
    
    # choose the next reaction 
    
        # initialise S to sum(a)
        
        # initialise mu to 0
        
        # initialise to N= r2*a0 - a[mu]

        # while loop:
        
            # while N is positive
            
            # add one to mu
            
            # substract a[mu] from N

        # when the while loop breaks, allocate the current value of mu to next_r

        # return T and next_r


# To debug choose_t_r write a function that runs choose_t_r 1000 times:

# define function run_choose_t_r_many_times with input a, the array of ai's

    # define a0 as the sum of a
    
    # define a zeros array r with dimensions 1000,1, to store the outputs of each time choose_t_r is run
    
    # define a for loop with index i that ranges from 0-999

        # define a variable res that is the output of choose_t_r

        # store the choice of reaction, res, in r
    
    # plot a histogram of all the reactions chosen, using pyplot hist

        # make sure that the bins are centred at the integer values that correspond 
        # to the reactions by defining an array bins with the bounds of each bin
    
 
# Ask youself what you expect the histogram to look like, given the current parameter values:
    # Run function run_choose_t_r_many_times with different values of a:
        # By changing the values of a you can check that different reactions are being chosen 
        # representatively with the system.
        # For example, for a = np.array([9000*0.0001, 10*1]), i.e. the rate for reaction R1 
        # is 0.0001 and for reaction R2 is 1, the proportions of choosing reaction R1 or R2 
        # should be similar, around 1:1, but for a = np.array([9000*0.01, 10*1]) R1 should 
        # be chosen many more times than R2, in fact, on a ratio of around 10:1.

#### Automate the calculation of the propensitites of the reactions

Think about how exactly to calculate the algebraic statement that equals `ai = hi*ci`.

It is helpful to write down many examples in order to come up with a formula.

Let's think through this particular example together:

We have the substrate stoichiometry being

```
 -2 -1 0
 -1  0 0
```

suppose we want to work out `hi` for `R1`. Let's start with `X1`. We know that the algebraic expression is `X1(X1-1)`. Notice that this is the same as the binomial coefficient `binom(X1,2)` multiplied by `2!`.

Now suppose that the substrate stoichiometry is

```
 -3 -1 0
 -1  0 0
```

to calculate the algebraic expression for `X1`, we know it is `X1*(X1-1)*(X1-2)`, which equals `binom(X1,3)*3!`.

We star to see a pattern here, can you write it down explicitly in Python language? The answer is below, but we encourage you to work it out for yourself. You will need the `binomial` and `factorial` functions. Also, be careful with the signs and remember that you can use the `absolute` function.

.

.

.

.

.

.

.

.

If you did the hand calculations, you saw that to calculate `hi` you need the following formula:

```
hi = hi*binom(current_species[j],np.absolute(stoch_subst[i,j]))*factorial(np.absolute(stoch_subst[i,j]))
```

Unfortunatelly, python does not seem to have a binomial coefficient function, so we define it here (perhaps it is fortunate, it gives us a chance to practice writting a new function! Make sure you understand it). (If you know of an already existing function, let us know!)

In [ ]:
# define a function to calculate the binomial coefficient
def binom(n,m):
    b=[0]*(n+1)
    b[0]=1
    for i in xrange(1,n+1):
        b[i]=1
        j=i-1
        while j>0:
            b[j]+=b[j-1]
            j-=1
    return b[m]

# debug - test binom


#### Write the main loop of the algorithm - Steps 1, 2 and 3

In [ ]:
# Now we write the main loop. Remember it is important to print values and using 
# the pause function to help yourself to debug the program.

# while loop  react_count < nrmax 
    
    # ****************************   
    # step 1: calculate ai and a0
    # ****************************   
    
    # debug
    print "step 1: calculate ai and a0"
    
    # initialise variable a to a ones array of dimensions num_rxn,1
    
    # (recall i ranges 1 -> M, where M is the number of reactions, j ranges 1 -> N, 
    # where N is the number of species)
    
    # make a for loop with index i that ranges from 0 to num_rxns
    
        # initialise hi to 1

        # make a for loop with index j that ranges from 0 to the number of species 
        # (i.e. len(init))
            
            # check the reactant (i,j) of the substrate stoichiomety matrix is involved 
            # in this reaction (use an if statement). 
                # if not, continue to the next reactant   

                # if yes, follow the steps below
                    
                    # check the reactant has molecules available (use an if statement)
                        # if it does not, equate hi to 0 (why?) and go to the next reactant
                
                        # if it does, calculate hi using the formula you worked out above.
                        # at this stage, for debugging purposes, you might like to include
                        # print statements of various values and using the pause function.

                            # e.g. debug
                    #print "reaction (i):", i
                    #print "reactant (j):", j
                    #print "current_species[j]:", current_species[j]
                    #print "stoch_subst[i,j]:", stoch_subst[i,j]
                    #print "np.absolute(stoch_subst[i,j]):", np.absolute(stoch_subst[i,j])
                    #print "binom(current_species[j],np.absolute(stoch_subst[i,j]):", binom(current_species[j],np.absolute(stoch_subst[i,j]))            
                    #print "factorial(np.absolute(stoch_subst[i,j])):", factorial(np.absolute(stoch_subst[i,j]))
                    #print "int(factorial(np.absolute(stoch_subst[i,j]))):", int(factorial(np.absolute(stoch_subst[i,j])))
                    #print "binom(current_species[j],np.absolute(stoch_subst[i,j]))*factorial(np.absolute(stoch_subst[i,j])):", binom(current_species[j],np.absolute(stoch_subst[i,j]))*factorial(np.absolute(stoch_subst[i,j]))            
                    #pause()
                    
        # save the current value of ai as hi*ci (i.e. hi*rates[i]) to variable a, at a[i]
                
    # save a0 as the sum of all a's

    
    # ****************************   
    # step 2: choose next t and r
    # ****************************   

    # debug
    print "step 2: choose next t and r"

    # run choose_t_r to get the change in time and the next reaction, save them to 
    # variables dt and next_r
    
    
    # ****************************   
    # step 3: update and store system
    # ****************************   
    
    # debug
    print "step 3: update and store system"
                          
    # update the system: 
    
        # update current_t by adding T
    
        # update current_species by adding np.transpose(stoch[next_r,:]), why?

        # update the time counter t_count and reaction counter react_count by adding 1
    
    # store current system to store_t[t_count], store_mols[t_count,:], store_r[t_count]

#### Collect and plot results of simulations

In [ ]:
# Get rid of empty entries in store_t, store_mols and store_r
                          
# plot the results of the simulation using pyplot, time on the y-axis and concentrations
# on the x-axis, remember to add a legend and to label the axes

# debug: plot a histogram of the reactions chosen by the algorithm (by plotting store_r)

### Part 3.C. Make the algorithm into a function so that you can run it with *any* biochemical system

Now, it would be great if we could just call a function that would work for any set of reactions and that we can just run one line of code rather than the whole set up and while loop over and over. We do this below. (Note that you can now get rid of the debugging commands as we have tested every bit and it is all working and we want the program to run smoothly without user input.)

#### Program the algorithm as a function

In [ ]:
# ----------------------------
# ----------------------------
# IMPORT ALL NECESSARY LIBRARIES AND MODULES
# ----------------------------
# ----------------------------


# ----------------------------
# ----------------------------
# DEFINE OTHER FUNCTIONS NECESSARY
# ----------------------------
# ----------------------------

# define a function to choose the next time and reaction


# define a function to calculate the binomial coefficient


# ----------------------------
# ----------------------------
# DEFINE SIMULATION FUNCTION
# ----------------------------
# ----------------------------

# define my_gillespie function with unput values init, rates, stoch_subst, stoch_prods, tmax, nrmax 
# and output values store_t, store_mols, store_r
    
    
    # --------------------------
    # set up
    # --------------------------
    
    # ****************************   
    # step 0: input rate values, initial contidions values and initialise time and reactions counter
    # ****************************   
    
    # define the stoichiometry (variable stoch) of the system as a numpy array as a sum of the substrate and product stoichiometries
    
    # initialise num_rxn and num_spec to the correct values
    
    # initialise current time and current species variables and the time and reaction counters, current_t, 
    # current_species, t_count, react_count
    
    # initialise variables to store time and molecule numbers, largenum, store_t, store_mols, store_r
    
    # store current time and state of system in store_t and store_mols
    
    
    # --------------------------
    # main while loop
    # --------------------------
    
        # ****************************   
        # step 1: calculate ai and a0
        # ****************************   
        
        # initialise variable a to a ones array of dimensions num_rxn,1
        
        # make a for loop with index i that ranges from 0 to num_rxns
    
            # initialise hi to 1

            # make a for loop with index j that ranges from 0 to the number of species (i.e. len(init))
            
                # check the reactant (i,j) of the substrate stoichiomety matrix is involved 
                # in this reaction (use an if statement). 
                
                # if not, continue to the next reactant   

                # if yes, follow the steps below
                    
                    # check the reactant has molecules available (use an if statement)
                        # if it does not, equate hi to 0 (why?) and go to the next reactant
                
                        # if it does, calculate hi using the formula you worked out above.
                        # at this stage, for debugging purposes, you might like to include
                        # print statements of various values and using the pause function.
                    
            # save the current value of ai as hi*ci (i.e. hi*rates[i]) to variable a, at a[i]
                
        # save a0 as the sum of all a's
        
        # ****************************   
        # step 2: choose next t and r
        # ****************************   

        # run choose_t_r to get the change in time and the next reaction, save them to 
        # variables T and next_r                          
        
        # ****************************   
        # step 3: update and store system
        # ****************************   
    
        # update the system: 
    
            # update current_t by adding T
    
            # update current_species by adding np.transpose(stoch[next_r,:]), why?

            # update the time counter t_count and reaction counter react_count by adding 1
    
        # store current system to store_t[t_count], store_mols[t_count,:], store_r[t_count]
    
    

    # store final output - get rid of empty entries in store_t, store_mols, store_r
    
    # return result

#### Run a few simulations with different scenarios

In [ ]:
# ----------------------------
# ----------------------------
# RUN SIMULATION
# R1: X1 + X2 -> 2X1
# R2: X1 -> 0
# ----------------------------
# ----------------------------

# ****************************
# step A: define rate values, initial contidions values, tmax and nrmax
# ****************************

# define the stochiometry of the substrates and products separatelly

# define the ci parameters (variable rates)

# define the initial conditions of the reactants (variable init)

# define the maximum time, tmax, and and maximum number of reactions, nrmax

# ****************************
# step B: run simulation
# ****************************

# ****************************
# step C: plot results of simulation in a graph
# ****************************



In [ ]:
# ----------------------------
# ----------------------------
# RUN ANOTHER SIMULATION
# R1: D -> D + R
# R2: R -> R + P
# R3: R -> 0
# R4: P -> 0
# ----------------------------
# ----------------------------

## Part 4. Next steps

Probably the bext thing you can try to do it to compare the results of these stochastic simulations with results you would get when you model the same systems deterministically, with a system of Ordinary Differential Equations (ODEs).

Next, as we have metioned before, the algorithm that we have written here is not the most computationally speedy and there are modifications of the Gillespie algorithm that are much more efficient. You could have a look at the difference in efficiency with the reaction method (Gibson & Bruck) and the tau-leaping method.

This is the end of the tutorial. Good job on finishing it! 